In [2]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved to /users/klee12/.cache/huggingface/token
Login successfu

In [3]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset("andstor/smart_contract_code_comments", 'default')

# Print basic information about the dataset
print(dataset)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 1267441
    })
    validation: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 130849
    })
    test: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_docume

# Training Dataset

In [4]:
train = dataset['train']

## Split Full Code and Code Comment

In [5]:
remove_cols = ['file_path', 'contract_address', 'language', 'class_name', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']

In [76]:
code = train.remove_columns(remove_cols)

In [77]:
# drop duplicates
full_code = pd.DataFrame(code)
full_code.drop_duplicates(inplace=True)

## Function Headers and Function Comments

In [22]:
remove_func_col = ['file_path', 'contract_address', 'class_code', 'class_documentation', 'language', 'class_name', 'class_documentation_type', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']

In [161]:
func_code = train.remove_columns(remove_func_col)

In [171]:
func = pd.DataFrame(func_code)

In [172]:
import re
count = 0

# Define the function that extracts function headers
def extract_function_header(func_code):
    # Regular expression to match Solidity function declarations
    func_header_regex = r'function\s*[\w\d_]*\s*\([^)]*\)\s*[^{]*'
    # Find all matches in the func_code
    matches = re.findall(func_header_regex, func_code)
    # Assume each func_code contains one function and take the first match
    # If there are multiple matches, this will need adjustment
    return matches[0] if matches else None

# Assume 'data' is your DataFrame loaded from CSV and it has a column 'func_code'
# You can apply the function directly on the DataFrame column
func['func_header'] = func['func_code'].apply(extract_function_header)

# filter out rows with no function headers
func = func[func['func_header'].apply(lambda x: x is not None and len(x) > 0)]

In [173]:
grouped_data = func.groupby('contract_name').agg({
    'func_header': list,
    'func_documentation': list
}).reset_index()

In [174]:
print(grouped_data)

      contract_name                                        func_header  \
0        $$$$$Token  [function totalSupply() public constant return...   
1               $33  [function balanceOf(address account) external ...   
2             $4RFI  [function totalSupply() external view returns ...   
3              $AVG  [function totalSupply() public constant return...   
4          $BANKINU  [function name() external view returns (string...   
...             ...                                                ...   
60688   zombittoken  [function totalSupply() constant returns (uint...   
60689     zp_minter  [function supportsInterface(bytes4 interfaceId...   
60690  zrankadictos  [function TokenERC20(\r\n    uint256 initialSu...   
60691       zsToken  [function deposit(uint256 amount, uint256 _tok...   
60692     zukiducks  [function tokenURI(uint256 _tokenId)\r\n    pu...   

                                      func_documentation  
0      [// -----------------------------------------

## Join On Contract Name

In [175]:
cleaned_data = pd.merge(grouped_data, full_code, on='contract_name', how='left')
cleaned_data = cleaned_data.drop_duplicates(subset=['contract_name'], keep='first')


In [176]:
print(cleaned_data.iloc[0])

contract_name                                                 $$$$$Token
func_header            [function totalSupply() public constant return...
func_documentation     [// ------------------------------------------...
class_code             contract $$$$$Token is ERC20Interface, SafeMat...
class_documentation    /**\r\n ERC20 Token, with the addition of symb...
Name: 0, dtype: object


# Generate Prompts into JSON File

Format:
- Task Description: `class_documentation` - contract comment
- Key Functions: 
  - `func_header, func_documentation`: function header and comment
- Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).

Example:

- Task Description: Create a token according to the ERC-20 standard with a mechanism for freezing assets for a specified period.
- Key Functions: 
  - `mint(address recipient, uint256 amount)`: Mints tokens to a specified address.
  - `freeze(address target, uint256 untilTimestamp)`: Freezes the assets of a specified address until a certain timestamp.
  - `transfer(address recipient, uint256 amount)`: Allows token transfers, checking if the sender's assets are frozen.
- Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).

In [177]:
# get 100 examples to use (TEST WITH 10)
raw = cleaned_data[:10]

In [231]:
def write_user_prompt(class_documentation, func_code_list, func_comment_list):
    prompt = 'Generate a smart contract using Solidity 0.8.0 that accomplishes the following requirements:\n'
    
    prompt += 'Task Description: {}\n'.format(class_documentation.replace("/", "").replace("*", ""))
    prompt += 'Key Functions:\n'
    
    for idx in range(len(func_code_list)):
        comment =  func_comment_list[idx].replace("/", "").replace("-", "").replace("\n", "")
        prompt += '{}: {}\n'.format(func_code_list[idx], comment)
    
    prompt += 'Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).\n'
    
    return prompt

In [192]:
test = raw.iloc[0]

with open('prompt-output.txt', 'w') as writefile:
    writefile.write(write_user_prompt(test['class_documentation'], test['func_header'], test['func_documentation']))
    

In [232]:
output = []

for idx, row in raw.iterrows():
    prompt = [{"role": "system", "content": "DEFI is an experienced Ethereum developer that writes properly functioning Solidity contracts."}]
    prompt.append({"role":"user","content":"{}".format(write_user_prompt(row['class_documentation'], row['func_header'], row['func_documentation']))})
    prompt.append({"role": "assistant", "content": "{}".format(row['class_code'])})
        
    output.append({"messages":prompt})

In [234]:
import jsonlines

with jsonlines.open('prompts.jsonl', mode='w') as writer:
    writer.write_all(output)